In [2]:
from docx import Document
from pandas import DataFrame, concat
import pandas as pd
import re
import os
import sqlite3

In [3]:
def read_docx(file_path):
    doc = Document(file_path)
    return doc

In [4]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)


In [10]:
def df_find_cases(file_path):
    words_for_table = []  # Use a list to store repeated entries
    sujeito_list = [] # Initialize sujeito_list outside the loop
    curso_list = []  # Initialize curso_list outside the loop
    sexo_list = [] # Initialize sexo_list outside the loop
    ano_list = [] # Initialize ano_list outside the loop
    cidade_list = [] # Initialize cidade_list outside the loop

    # Iterate through all files in the specified directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".docx"):
            file_path = os.path.join(directory_path, filename)
            text = read_docx(file_path)

            ### WORDS WITH PATTERN ###
            pattern = r'\S*\[[^\]]*(?:ɾ|ɹ|h|∅r)[^\]]*\]\S*'
            matches = re.findall(pattern, text)
            words_for_table.extend(matches)

            ###### SUJEITOS #######
            sujeito_match = re.search(r'\bSUJEITO:\s*(\S+)', text)
            if sujeito_match:
                sujeito_list.extend([sujeito_match.group(1)] * len(matches))
            else:
                sujeito_list.extend(['Not Specified'] * len(matches))
            
             ##### CURSO #########
            curso_match = re.search(r'\bCURSO:\s*(\S+)', text)
            if curso_match:
                curso_list.extend([curso_match.group(1)] * len(matches))
            else:
                curso_list.extend(['Not Specified'] * len(matches))

             ##### SEXO #######
            sexo_match = re.search(r'\bSEXO:\s*(\S+)', text)
            if sexo_match:
                sexo_list.extend([sexo_match.group(1)] * len(matches))
            else:
                sexo_list.extend(['Not Specified'] * len(matches))

            
             ####### ANO ########
            ano_match = re.search(r'\bANO:\s*(\S+)', text)
            if ano_match:
                ano_list.extend([ano_match.group(1)] * len(matches))
            else:
                ano_list.extend(['Not Specified'] * len(matches))

            ####### CIDADE ########
            cidade_match = re.search(r'\bCIDADE:\s*(\S+)', text)
            if cidade_match:
                cidade_list.extend([cidade_match.group(1)] * len(matches))
            else:
                cidade_list.extend(['Not Specified'] * len(matches))
    
    df = DataFrame({'palavra': words_for_table, 'sujeito': sujeito_list, 'curso': curso_list, 'sexo': sexo_list, 'ano': ano_list, 'cidade': cidade_list})


    #remove commas and periods at the end of values
    punctuation_pattern = r'[.,…!?]+$'
    a_pattern = r'-a'
    o_pattern = r'-o'
    pattern = punctuation_pattern + '|' + a_pattern + '|' + o_pattern
    df['palavra'] = df['palavra'].str.replace(pattern, '', regex=True)
        
            
    return df

In [13]:
def create_verbo_ou_nome(row, i):
    
    pattern_n = r'\[[^\]]*\(n\)[^\]]*\]'
    pattern_v = r'\[[^\]]*\(v\)[^\]]*\]'

    pattern = pattern_n + '|' + pattern_v

    verbo_ou_nome_cru = re.findall(pattern, row['palavra'])
    
    if row['verbo_ou_nome'] == '':
        
        if re.search(pattern_n, verbo_ou_nome_cru[i]):
            verbo_ou_nome = 'nome'
        if re.search(pattern_v, verbo_ou_nome_cru[i]):
            verbo_ou_nome ='verbo'    

    return verbo_ou_nome

In [14]:
def apply_cols(directory_path):
    df = df_find_cases(directory_path)

    df['verbo_ou_nome'] = ''
    df['verbo_ou_nome'] = df.apply(lambda row: create_verbo_ou_nome(row, 0), axis=1)
    # df['patalizacao_cru'] = ''
    # df['patalizacao_cru'] = df.apply(lambda row: create_patalizacao_cru(row, 0), axis=1)
    
    # df['sonoridade'] = df['patalização'].apply(determine_sonoridade)

    # df['vogais_alvo'] = ''
    # df['vogais_alvo'] = df.apply(lambda row: create_vogais_alvo(row, 0), axis=1)
    # df = df.apply(create_posicao_patalizacao, axis=1)
    # df = df_add_duplicated_matches(df)

    # df['patalização'] = df.apply(lambda row: create_patalizacao(row, 1), axis=1)
    # df['patalizacao_cru'] = df.apply(lambda row: create_patalizacao_cru(row, 1), axis=1)
    # df['sonoridade'] = df['patalização'].apply(determine_sonoridade)
    # df['vogais_alvo'] = df.apply(lambda row: create_vogais_alvo(row, 1), axis=1)

    # df = df.apply(final_create_posicao_patalizacao, axis=1)
    # df = df.drop(columns=['copy_palavra'])

    
    return df


In [15]:
directory_path = "../data/"
df = apply_cols(directory_path)
df

,palavra,sujeito,curso,sexo,ano,cidade,verbo_ou_nome
0,pe[ɹ(n)]gunta,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,nome
1,cu[ɹ(n)]so,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,nome
2,convive[∅r(v)],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,verbo
3,se[ɹ(v)],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,verbo
4,melho[∅r(n)],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,nome
...,...,...,...,...,...,...,...
1878,es[tʃɪ]ve[ɹ(n)][dʒɪ]s,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,nome
1879,acaba[∅r(v)],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,verbo
1880,pate[ɹ(n)]no,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,nome
1881,fo[ɹ(n)][tʃɪ][s],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,nome


In [17]:
word = "aprende[∅(v)]"
adjusted_length = 0
inside_brackets = False

for char in word:
    if char == '[':
        inside_brackets = True
        adjusted_length += 1
    elif char == ']':
        inside_brackets = False
    elif inside_brackets:
        # Inside brackets, count as one character
        continue
    else:
        # Outside brackets, count as individual characters
        adjusted_length += 1

print("Adjusted number of characters:", adjusted_length)


Adjusted number of characters: 8


In [8]:
def create_verbo_ou_nome(row, i):
    
    pattern_n = r'\[[^\]]*(n)[^\]]*\]'
    pattern_v = r'\[[^\]]*(v)[^\]]*\]'

    pattern = pattern_n + '|' + pattern_v

    verbo_ou_nome_cru = re.findall(pattern, row['palavra'])
    
    

    if re.search(pattern_n, patalizacao_cru[0]):
        patalização = '[t]'
    if re.search(pattern_tf, patalizacao_cru[0]):
        patalização ='[tʃ]'
    if re.search(pattern_dz, patalizacao_cru[0]):
        patalização = '[dʒ]'
    if re.search(pattern_de, patalizacao_cru[0]):
        patalização = '[de]'
    if re.search(pattern_di, patalizacao_cru[0]):
        patalização = '[dɪ]'

    return patalizacao_cru, patalização

In [9]:
# Function to determine 'sonoridade' based on 'patalização' values
def determine_sonoridade(row):
    
    if row == '[t]' or row == '[tʃ]':
        return 'surda'
    elif row == '[de]' or row == '[dɪ]' or row == '[dʒ]':
        return 'sonora'
    
    return None


In [10]:

def create_vogais_alvo(row):
    vogais_alvo = []
    
    pattern_I = r'\[[^\]]*ɪ[^\]]*\]'
    pattern_e = r'\[[^\]]*e[^\]]*\]'
    pattern = pattern_I + '|' + pattern_e

    match = re.search(pattern, row['palavra'])

    if match:
        vogais_cru = match.group(0)

        if re.search(pattern_I, vogais_cru):
            return '[ɪ]'
        if re.search(pattern_e, vogais_cru):
            return '[e]'
        
    return None
    
    

In [11]:
def apply_cols(file_path):
    df = df_find_cases_without_duplication(file_path)
    df['patalizacao_cru'], df['patalização'] = zip(*df.apply(create_patalizacao, axis=1))
    
    # df['patalização_count'] = df['patalização'].apply(lambda x: len([item.strip() for item in x.split(',')]))
    df['sonoridade'] = df['patalização'].apply(determine_sonoridade)
    df['vogais_alvo'] = df.apply(create_vogais_alvo, axis=1)
    return df

In [12]:
df = apply_cols(file_path)
df

,palavra,patalizacao_cru,patalização,sonoridade,vogais_alvo
0,gran[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ]
1,[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ]
2,[dʒɪ]reito,[[dʒɪ]],[dʒ],sonora,[ɪ]
3,úl[tʃɪ]mo,[[tʃɪ]],[tʃ],surda,[ɪ]
4,realmen[tʃɪ],[[tʃɪ]],[tʃ],surda,[ɪ]
...,...,...,...,...,...
172,es[tɪ]ve[ɹ(v)][de][s],"[[tɪ], [de]]",[t],surda,[ɪ]
173,[de],[[de]],[de],sonora,[e]
174,[dɪ]ze[ɹ(v)],[[dɪ]],[dɪ],sonora,[ɪ]
175,fo[ɹ(n)][te][s],[[te]],[t],surda,[e]


In [13]:
def create_posicao_patalizacao_final(row):
    
    
    unico_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[[^\]]*\]$'

  
    comeco_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]'

   
    meio_pattern = r'\[[^\]]*(tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]'
    meio_pattern_mais = r'\S\[[^\]]*(?:tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]\S'


    final_pattern = r'\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]$'


    

    row['copy_palavra'] = row['palavra']

    if re.search(unico_pattern, row['palavra']):
        row['posição_patalização'] = row['patalização'] + 'único'
        row['copy_palavra'] = re.sub(unico_pattern, '', row['copy_palavra'])

    elif re.search(comeco_pattern, row['palavra']):
        row['posição_patalização'] = row['patalização'] + ' comeco'
        row['copy_palavra'] = re.sub(comeco_pattern, '', row['copy_palavra'])

    elif re.search(meio_pattern_mais, row['palavra']):
        row['posição_patalização'] = row['patalização'] + ' meio'
        row['copy_palavra'] = re.sub(meio_pattern, '', row['copy_palavra'], count=1)
    
    elif re.search(final_pattern, row['palavra']):
        row['posição_patalização'] = row['patalização'] + ' final'
        row['copy_palavra'] = re.sub(final_pattern, '', row['copy_palavra'])
    
    else:
        row['posição_patalização'] = ''
    return row


df = df.apply(create_posicao_patalizacao_final, axis=1)
df


/tmp/ipykernel_7086/3712872512.py:21: FutureWarning: Possible nested set at position 30
  if re.search(unico_pattern, row['palavra']):


,palavra,patalizacao_cru,patalização,sonoridade,vogais_alvo,copy_palavra,posição_patalização
0,gran[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],gran,[dʒ] final
1,[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],,[dʒ] comeco
2,[dʒɪ]reito,[[dʒɪ]],[dʒ],sonora,[ɪ],reito,[dʒ] comeco
3,úl[tʃɪ]mo,[[tʃɪ]],[tʃ],surda,[ɪ],úlmo,[tʃ] meio
4,realmen[tʃɪ],[[tʃɪ]],[tʃ],surda,[ɪ],realmen,[tʃ] final
...,...,...,...,...,...,...,...
172,es[tɪ]ve[ɹ(v)][de][s],"[[tɪ], [de]]",[t],surda,[ɪ],esve[ɹ(v)][de][s],[t] meio
173,[de],[[de]],[de],sonora,[e],,[de]único
174,[dɪ]ze[ɹ(v)],[[dɪ]],[dɪ],sonora,[ɪ],ze[ɹ(v)],[dɪ] comeco
175,fo[ɹ(n)][te][s],[[te]],[t],surda,[e],fo[ɹ(n)][s],[t] meio


In [17]:
pd.set_option('display.max_rows', None)

In [18]:
df

,palavra,patalizacao_cru,patalização,sonoridade,vogais_alvo,copy_palavra,posição_patalização
0,gran[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],gran,[dʒ] final
1,[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],,[dʒ] comeco
2,[dʒɪ]reito,[[dʒɪ]],[dʒ],sonora,[ɪ],reito,[dʒ] comeco
3,úl[tʃɪ]mo,[[tʃɪ]],[tʃ],surda,[ɪ],úlmo,[tʃ] meio
4,realmen[tʃɪ],[[tʃɪ]],[tʃ],surda,[ɪ],realmen,[tʃ] final
5,an[tɪ]s,[[tɪ]],[t],surda,[ɪ],ans,[t] meio
6,[dɪ],[[dɪ]],[dɪ],sonora,[ɪ],,[dɪ]único
7,Expecta[tʃɪ]va[s],[[tʃɪ]],[tʃ],surda,[ɪ],Expectava[s],[tʃ] meio
8,[dʒɪ]sso,[[dʒɪ]],[dʒ],sonora,[ɪ],sso,[dʒ] comeco
9,basicamen[tʃɪ],[[tʃɪ]],[tʃ],surda,[ɪ],basicamen,[tʃ] final


In [22]:
df = apply_cols(file_path)
df = df.apply(create_posicao_patalizacao_final, axis=1)

In [23]:
def df_add_duplicated_matches(file_path):
    df = apply_cols(file_path)
    df = df.apply(create_posicao_patalizacao_final, axis=1)

    # Regular expression pattern
    pattern = r'\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]'

    
    # Find matches for each value in the column
    matches = df['palavra'].str.findall(pattern)
    
    # idenfitfy idexes where there is a match
    match_index = df[matches.apply(lambda x: len(x) > 1)].index
    
    #turn index object to list
    match_list = list(match_index)
    
    i = 0
    for element in match_list:

        element_used = element + i
        new_row_data = df.loc[element_used:element_used]
        new_row_data = new_row_data[['palavra','copy_palavra', 'patalizacao_cru']]

        df = concat([df.iloc[:element_used + 1], new_row_data, df.iloc[element_used + 1:]]).reset_index(drop=True)
        i = i + 1
    
    df = df.fillna('') 
    return df
# pd.set_option('display.maxs_rows', None)
df = df_add_duplicated_matches(file_path)


In [24]:
def new_create_patalizacao(row, i):
    
    pattern_t = r'\[[^\]]*t(?!ʃ)[^\]]*\]'
    pattern_tf = r'\[[^\]]*tʃ[^\]]*\]'
    pattern_dz = r'\[[^\]]*dʒ[^\]]*\]'
    pattern_de = r'\[[^\]]*de[^\]]*\]'
    pattern_di = r'\[[^\]]*dɪ[^\]]*\]'

    pattern = pattern_t + '|' + pattern_tf + '|' + pattern_dz + '|' + pattern_de + '|' + pattern_di
    
    
    patalizacao_cru = re.findall(pattern, row['palavra'])

    if row['patalização'] == '':
        if re.search(pattern_t, patalizacao_cru[i]):
            return '[t]'
        if re.search(pattern_tf, patalizacao_cru[i]):
            return '[tʃ]'
        if re.search(pattern_dz, patalizacao_cru[i]):
            return '[dʒ]'
        if re.search(pattern_de, patalizacao_cru[i]):
            return '[de]'
        if re.search(pattern_di, patalizacao_cru[i]):
            return '[dɪ]'

    return row['patalização']

# df['patalização'] = df.apply(new_create_patalizacao, axis=1)
df['patalização'] = df.apply(lambda row: new_create_patalizacao(row, 1), axis=1)

In [25]:
df['sonoridade'] = df['patalização'].apply(determine_sonoridade)


In [63]:
def new_create_vogais_alvo(row):
    vogais_alvo = []
    
    pattern_I = r'\[[^\]]*ɪ[^\]]*\]'
    pattern_e = r'\[[^\]]*e[^\]]*\]'

    patalizacao_cru = row['patalizacao_cru']

    if row['vogais_alvo'] == '':
        if re.search(pattern_I, patalizacao_cru[1]):
            return '[ɪ]'
        if re.search(pattern_e, patalizacao_cru[1]):
            return '[e]'
        
    return row['vogais_alvo']
    

In [64]:
df['vogais_alvo'] = df.apply(create_vogais_alvo, axis=1)

In [26]:
def new_create_posicao_patalizacao_final(row):
    
    
    unico_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[[^\]]*\]$'

  
    comeco_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]'

   
    meio_pattern = r'\[[^\]]*(tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]'
    meio_pattern_mais = r'\S\[[^\]]*(?:tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]\S'


    final_pattern = r'\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]$'


    

    if row['posição_patalização'] == '':


        if re.search(comeco_pattern, row['copy_palavra']) or re.search(meio_pattern_mais, row['copy_palavra']):
            row['posição_patalização'] = row['patalização'] + ' meio'

        elif re.search(final_pattern, row['copy_palavra']):
            row['posição_patalização'] = row['patalização'] + ' final'
    return row


df = df.apply(new_create_posicao_patalizacao_final, axis=1)


In [27]:
df.drop(columns=['patalizacao_cru', 'copy_palavra'])

,palavra,patalização,sonoridade,vogais_alvo,posição_patalização
0,gran[dʒɪ],[dʒ],sonora,[ɪ],[dʒ] final
1,[dʒɪ],[dʒ],sonora,[ɪ],[dʒ] comeco
2,[dʒɪ]reito,[dʒ],sonora,[ɪ],[dʒ] comeco
3,úl[tʃɪ]mo,[tʃ],surda,[ɪ],[tʃ] meio
4,realmen[tʃɪ],[tʃ],surda,[ɪ],[tʃ] final
5,an[tɪ]s,[t],surda,[ɪ],[t] meio
6,[dɪ],[dɪ],sonora,[ɪ],[dɪ]único
7,Expecta[tʃɪ]va[s],[tʃ],surda,[ɪ],[tʃ] meio
8,[dʒɪ]sso,[dʒ],sonora,[ɪ],[dʒ] comeco
9,basicamen[tʃɪ],[tʃ],surda,[ɪ],[tʃ] final
